In [ ]:
####################### You have to run this cell #######################

% load_ext autoreload
% autoreload 2
import sys, os
import os
cwd = os.getcwd()
path = os.path.join(cwd, os.pardir)
sys.path.append(path)


# Put all imports below

In [2]:
from models import User,Test,Payments,UserRoles,Role,College,CollegeTest,QuestionAttempt,SectionAttempt,TestAttempt,Question,Choice,Section
from models.Package import PackageTypes
from app import create_app
from extensions import db
from sqlalchemy import and_,outerjoin,or_
from sqlalchemy.orm import contains_eager, joinedload,load_only
from pprint import pprint

In [5]:
app = create_app('./configs/prod.ini')
app.app_context().push()

## Simple Queries 
For very simple queries, use MODEL.query , followed by filter arguments 

In [6]:
result = User.query.filter(User.id ==1).first()

pprint(result.todict(['full_name', # can specify which arguments to serialize
             'id',
             'referral_bonus',
             'referral_code',
             'wallet','college']) )

{'college': {'college_logo': None, 'college_name': 'Demo College', 'id': 1},
 'full_name': 'Sayantan Chatterjee',
 'id': 1,
 'referral_bonus': None,
 'referral_code': None,
 'wallet': 0}


In [7]:
db.session.close_all() # this is required for just this notebook 

In [8]:
pprint(result)

User({
    "college": {
        "college_logo": null,
        "college_name": "Demo College",
        "id": 1
    },
    "college_id": 1,
    "email": "sayantan.chat@yahoo.co.in",
    "fb_id": "10205780376428005",
    "fb_oauth_token": "EAARlyK0lHg4BAL9XZCdmcPiSg1ICdPowoqDux0POQnMreIOWdKFzTXK7Pqkw8NswWDIZCzuPtjx86QbTtZC6Wnz7kDA7ZBtvwBhjQ986gkrrHVzWTO78jfXEs5MfJPk02B2pZA6dx6YZAiKllSl7tmuqqf9KUsZAxvya7n14LacZCgZDZD",
    "full_name": "Sayantan Chatterjee",
    "google_id": null,
    "google_oauth_token": null,
    "google_token_id": null,
    "id": 1,
    "is_active": true,
    "phone_no": null,
    "profile_picture": null,
    "referral_bonus": null,
    "referral_code": null,
    "referral_code_used": null,
    "type": "standard",
    "wallet": 0
})


## Queries with in clause 
To fetch all items that satisfy elements in a list

In [9]:
result = User.query.filter(User.id.in_([1,2,3])).all()


pprint([result.todict(['full_name', # notice how this is serialised 
             'id',
             'referral_bonus',
             'referral_code',
             'wallet',
             'college']) for result in result])

[{'college': {'college_logo': None, 'college_name': 'Demo College', 'id': 1},
  'full_name': 'Sayantan Chatterjee',
  'id': 1,
  'referral_bonus': None,
  'referral_code': None,
  'wallet': 0},
 {'college': None,
  'full_name': 'Rohan Ghosh',
  'id': 2,
  'referral_bonus': None,
  'referral_code': None,
  'wallet': 0},
 {'college': None,
  'full_name': 'Debraj Ghosh',
  'id': 3,
  'referral_bonus': None,
  'referral_code': None,
  'wallet': 0}]


In [10]:
db.session.close_all() # this is required for just this notebook 

## Queries with nested relationships
Sometimes, queries with nested relationships are required. 
An example is User -> payments 

It is possible to fetch user first, then do user.payments  , but this will result in two queries. Also , it is not possible to do filtering on the nested resource. 

In [11]:
# Get User with id =1 and all his payments 

result = db.session.query(User).filter(User.id==1).first()

In [12]:
pprint(result.payments) # will do a 2nd query 

[Payments({
    "id": 2,
    "paymentAmount": 10000,
    "paymentId": "pay_7VA9D4Yp6ugurp",
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 24,
    "userId": 1
}),
 Payments({
    "id": 550,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 58,
    "userId": 1
}),
 Payments({
    "id": 775,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 93,
    "userId": 1
}),
 Payments({
    "id": 888,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 92,
    "userId": 1
}),
 Payments({
    "id": 889,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 94,
    "userId": 1
}),
 Payments({
    "id": 890,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentS

In [13]:
db.session.close_all() # this is required for just this notebook 

## Better way of doing this

This will happen in one query. 
by adding a join and .options(contains_eager()), you are fooling sqlalchemy to think it has fetched all the things that it needs. 
Thus, calling test.payments will not cause a 2nd trip. 


In [14]:
# Get User with id =1 and all his payments 
result = User.query\
        .join(User.payments)\
        .options(contains_eager(User.payments))\
        .filter(User.id ==1).all()[0]
        
# notice that I did not call first() , because first() will literally add 
#limit 1 to the end of the SQL query. In that case, sqlalchemy will be                                             #unable to parse the multi row result into a user -> multi payments object

db.session.close_all() # this is required for just this notebook 

In [30]:
pprint(result.todict())

{'college_id': 1,
 'email': 'sayantan.chat@yahoo.co.in',
 'fb_id': '10205780376428005',
 'fb_oauth_token': 'EAARlyK0lHg4BAL9XZCdmcPiSg1ICdPowoqDux0POQnMreIOWdKFzTXK7Pqkw8NswWDIZCzuPtjx86QbTtZC6Wnz7kDA7ZBtvwBhjQ986gkrrHVzWTO78jfXEs5MfJPk02B2pZA6dx6YZAiKllSl7tmuqqf9KUsZAxvya7n14LacZCgZDZD',
 'full_name': 'Sayantan Chatterjee',
 'google_id': None,
 'google_oauth_token': None,
 'google_token_id': None,
 'id': 1,
 'is_active': True,
 'payments': [{'id': 2,
               'paymentAmount': 10000,
               'paymentId': 'pay_7VA9D4Yp6ugurp',
               'paymentPromoCode': None,
               'paymentSuccess': True,
               'test': {'character': 'Topic',
                        'created_date': '2016-11-22T22:54:18',
                        'id': 24,
                        'is_active': True,
                        'leaderboard_id': None,
                        'name': 'Reading Comprehension 01',
                        'price': 0,
                        'type': 'CAT'},
     

## Weird behaviour by `contains_eager()`

It is not possible to call `first()` in the previous example, because it will literally add `LIMIT 1`
to the query. This prevents sqlalchemy from parsing a potentially multi row result into a complex model object. 

However, calling `one()` does not explicitly add `LIMIT 1` to the query. It will still allow a multi row result to be returned , and parsed. 

The interesting thing is that , it will `one()` will restrict the primary object being searched for to exactly one object. 

It will throw an exception if multiple results for the primary object were returned

This somewhat helps when looking for only the first value. (it will still throw a multiple result exception if multiple rows for primary object is returned)



## Alternative, using joinedload

Calling first() on a joined query wil cause incorrect results, because sqlalchemy will literally add LIMIT 1 to the end of the query. 
In order to resolve this issue, there exists joinedload(). 
If joinedload is used instead of contains_eager, sqlalchemy first wraps the query in a sub query, and then applies LIMIT on that. 



In [16]:
result = User.query\
        .options(joinedload(User.payments))\
        .filter(User.id ==1).first()

db.session.close_all() # this is required for just this notebook 

In [17]:
pprint(result.todict(['payments',{'payments':'id'}]))

{'payments': [{'id': 2},
              {'id': 550},
              {'id': 775},
              {'id': 888},
              {'id': 889},
              {'id': 890},
              {'id': 891},
              {'id': 892},
              {'id': 893},
              {'id': 894},
              {'id': 895},
              {'id': 896},
              {'id': 908},
              {'id': 930},
              {'id': 17434}]}


## JoinedLoad vs Contains Eager 

JoinedLoad is designed to be simple, and not have the ability to alter results using conditions. 
If you have to perform conditions on some joined table, joinedload will not be helpful. 

Contains_eager is meant to be a 'short_circuit' mechanism. It is your way of telling sqlalchemy that this is all the results I want. Use contains_eager when performing complex intermediate queries

# Warning!

When using contains_eager and having something like 

`some_query.all()[0]`

**Be extra careful of the query.** 
The query may be fetching millions of records , and eventually you are using just the first one. 
One way to prevent such mistakes is to always assert that the length of the result is 1. 

**The safer alternative is to use .one() instead **

## Conditional 

Adding a condition on the nested resource

In [18]:
result = db.session.query(User) \
    .join(User.payments) \
    .options(contains_eager(User.payments)) \
    .filter(
        Payments.testId.in_([24, 58, 93, 4])) \
    .filter(User.id == 1).one_or_none()
# user.id =1 is guaranteed to have exactly 1 row in Users table, so one() should work. 
# if an exception occurs, theres a bigger problem and database admin should be notified.


In [19]:
db.session.close_all() # this is required for just this notebook 

In [20]:
pprint(result)

User({
    "college_id": 1,
    "email": "sayantan.chat@yahoo.co.in",
    "fb_id": "10205780376428005",
    "fb_oauth_token": "EAARlyK0lHg4BAL9XZCdmcPiSg1ICdPowoqDux0POQnMreIOWdKFzTXK7Pqkw8NswWDIZCzuPtjx86QbTtZC6Wnz7kDA7ZBtvwBhjQ986gkrrHVzWTO78jfXEs5MfJPk02B2pZA6dx6YZAiKllSl7tmuqqf9KUsZAxvya7n14LacZCgZDZD",
    "full_name": "Sayantan Chatterjee",
    "google_id": null,
    "google_oauth_token": null,
    "google_token_id": null,
    "id": 1,
    "is_active": true,
    "payments": [
        {
            "id": 2,
            "paymentAmount": 10000,
            "paymentId": "pay_7VA9D4Yp6ugurp",
            "paymentPromoCode": null,
            "paymentSuccess": true,
            "testId": 24,
            "userId": 1
        },
        {
            "id": 550,
            "paymentAmount": null,
            "paymentId": null,
            "paymentPromoCode": null,
            "paymentSuccess": true,
            "testId": 58,
            "userId": 1
        },
        {
            "id": 77

In [21]:
pprint(result.payments) # will not call db again. 

[Payments({
    "id": 2,
    "paymentAmount": 10000,
    "paymentId": "pay_7VA9D4Yp6ugurp",
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 24,
    "userId": 1
}),
 Payments({
    "id": 550,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 58,
    "userId": 1
}),
 Payments({
    "id": 775,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 93,
    "userId": 1
}),
 Payments({
    "id": 930,
    "paymentAmount": null,
    "paymentId": null,
    "paymentPromoCode": null,
    "paymentSuccess": true,
    "testId": 4,
    "userId": 1
})]


## Multi Nesting 

The same logic can be extended to arbitrary amount of nesting. 

In this example , we want to get a particular user's payments , and the name of the test that those payments belong to. 


In [22]:
# if you have multiple joins ,his is how you specify to load all(line 6-7)

result = db.session.query(User)\
        .join(User.payments)\
        .join(Payments.test)\
        .options(contains_eager(User.payments)\
                .contains_eager(Payments.test))\
        .filter(Payments.testId.in_([24,58,93,4]))\
        .filter(User.id ==1).all()[0] # notice this. same reason as previous example.
db.session.close_all() # this is required for just this notebook 

In [23]:
pprint([result.test.name for result in result.payments])  # did not do a 2nd db call

['Reading Comprehension 01', 'Beatest Mock', 'Beatest_Mock', 'BEAT CAT 02']


## A Real Example

Check if a user can access a particular question attempt.<br>This is done by doing a join from 
TestAttempt --> SectionAttempts --> QuestionAttempts 

A filter to match the userid with the TestAttempt's user id , 
and the inputs for the (SectionAttempt.id , Question.id).<br>
If user has access to that particular pair, a row will be returned, else nothing 
will be returned.

In [24]:
query = TestAttempt \
    .query \
    .join(SectionAttempt) \
    .join(QuestionAttempt) \
    .filter(QuestionAttempt.section_attempt_id == 12) \
    .filter(QuestionAttempt.question_id == 5689) \
    .filter(TestAttempt.user_id == 536) \
    .exists()  # this puts the rest of the query into a sub query, and adds exists() to the result

exists = db.session.query(query).scalar()
db.session.close_all()  # this is required for just this notebook


In [25]:
pprint(exists)

True


## Tip - use load_only to really speed up queries

load_only prevents loading all columns. Sometimes, you dont need all the columns, and doing this<br>can really improve speed. E.g. By using only primary keys, its possible to speed up query speed by 10x.  

In many cases, especially joins , only ids are required. In such cases, you can notice a dramatic speed boost

### Example 

In order to calculate score, it is possible to do this join: 

`TestAttempt -> Test -> Sections -> Questions -> Choices`

and also

`TestAttempt -> Test -> Sections -> Questions -> QuestionAttempt`

Test has instruction html (which is lots of data), and so does Question.
We do not need these html in any way 

In [26]:
def query_without_load_only():
    result = (TestAttempt.query
              .options(
            contains_eager(TestAttempt.test)
            .contains_eager(Test.sections)
            .contains_eager(Section.questions)
            .contains_eager(Question.choices))
              .options(
            contains_eager(TestAttempt.test)
            .contains_eager(Test.sections)
            .contains_eager(Section.questions)
            .contains_eager(Question.question_attempts))
              .join(TestAttempt.test)
              .join(Test.sections)
              .join(Section.questions)
              .join(Question.choices)
              .join(Question.question_attempts)
              .filter(TestAttempt.id == 1)
              .one())

    db.session.close_all()  # this is required for just this notebook 
    return result


In [27]:
%timeit -n 1 query_without_load_only()

661 ms ± 216 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
pprint(query_without_load_only())

In [32]:
def query_with_load_only():
    result = TestAttempt.query\
    .options(
        contains_eager(TestAttempt.test).load_only(Test.id)\
        .contains_eager(Test.sections).load_only(Section.id)\
        .contains_eager(Section.questions).load_only(
            Question.id,
            Question.points_correct,
            Question.points_wrong)\
            .contains_eager(Question.choices)\
                .load_only(Choice.id,Choice.is_correct)\
        )\
    .options(
        contains_eager(TestAttempt.test).load_only(Test.id)\
        .contains_eager(Test.sections).load_only(Section.id)\
        .contains_eager(Section.questions).load_only(Question.id)
        .contains_eager(Question.question_attempts)\
        )\
    .join(TestAttempt.test)\
    .join(Test.sections)\
    .join(Section.questions)\
    .join(Question.choices)\
    .join(Question.question_attempts)\
    .filter(TestAttempt.id == 1)\
    .one()
    
    db.session.close_all() # this is required for just this notebook 
    return result

In [33]:
%timeit -n 1 query_with_load_only()

613 ms ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
pprint(query_with_load_only())

#### Difference is 3.45 seconds vs 454 milliseconds. Over 7x faster! 